# Topic Modeling with [BERTopic](https://github.com/MaartenGr/BERTopic)
This notebook performs the Topic Modeling task using BERTopic.

In [ ]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sentence_transformers import SentenceTransformer
import nltk
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab',quiet=True)
nltk.download('punkt',quiet=True)
pass

/home/mrpio/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
df = pd.read_csv('../1-etl/reviews_clean.csv').fillna('')
corpus = df[df['stars']<=5]['content']
corpus = list(filter(lambda x:len(x)>20,corpus))
corpus = [sent for doc in corpus for sent in sent_tokenize(doc)]
len(corpus)

2821

In [5]:
# Create embeddings from the documents
sentence_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = sentence_model.encode(list(corpus))

In [14]:
# Define sub-models
vectorizer = CountVectorizer(stop_words="english")
umap_model = UMAP(n_neighbors=5, n_components=10,
                  min_dist=0.0, metric='cosine')
# hdbscan_model = HDBSCAN(min_cluster_size=10, min_samples=2, metric='euclidean')

# Train our topic model with BERTopic
topic_model = BERTopic(
    language="english",
    calculate_probabilities=True,
    verbose=True,
    embedding_model=sentence_model,
    # umap_model=umap_model,
    # hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer,
    n_gram_range=(1, 3)
)
topics, probs = topic_model.fit_transform(corpus, embeddings)
topic_model.get_topic_info()

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,1161,-1_the_to_it_and,"the, to, it","[the, to, it, and, printer, this, that, of, fo...","[Constantly ""offline"", disconnecting from my r..."
1,0,124,0_hp_with_support_with hp,"hp, with, support","[hp, with, support, with hp, hp and, and, amaz...",[I spent literal hours on the laptop with hp s...
2,1,83,1_service_customer_customer service_he,"service, customer, customer servic","[service, customer, customer service, he, me, ...",[When calling customer support they immediatel...
3,2,77,2_set_easy_set up_up,"set, easy, set up","[set, easy, set up, up, to set, setup, printer...",[This printer was vry easy to set up and conne...
4,3,74,3_quality_print quality_quality is_prints,"quality, print quality, quality is","[quality, print quality, quality is, prints, t...",[The print quality is not very good on this on...
5,4,74,4_tray_paper_paper tray_the paper,"tray, paper, paper tray","[tray, paper, paper tray, the paper, the, slid...",[The only thing I don’t like is the paper tray...
6,5,67,5_hp_hp printer_hp printers_printers,"hp, hp printer, hp printers","[hp, hp printer, hp printers, printers, years,...",[I bought this HP printer to replace a very si...
7,6,62,6_printer_this_printer is_for,"printer, this, printer is","[printer, this, printer is, for, good, this pr...",[Have not learned everything about it but so f...
8,7,49,7_scan_scanning_to scan_sided,"scan, scanning, to scan","[scan, scanning, to scan, sided, page, documen...",[The duplex function is useless: When trying t...
9,8,47,8_buy_would_price_buy it,"buy, would, price","[buy, would, price, buy it, recommend, would n...","[Would definitely buy again., Would definitely..."


In [ ]:
# topic_model.save('model.bertopic')

In [52]:
topics_to_merge = [[-1, 5, 6, 26, 45], [2, 9],
                   [13, 18, 20], [12, 21, 36, 40], [11, 16], [27, 4], [14, 50], [15,22]]
topic_model.merge_topics(corpus, topics_to_merge=topics_to_merge)

In [ ]:
results_df = topic_model.get_topic_info()
results_df.to_csv('topic_modeling.csv')
results_df

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,1325,-1_the_printer_to_it,outliers,"[the, printer, to, it, and, this, hp, for, is,...",[If you don't print much this printer is not f...
1,0,124,0_hp_with_support_and,support,"[hp, with, support, and, with hp, hp and, to, ...",[I spent literal hours on the laptop with hp s...
2,1,122,1_set_set up_easy_up,set up,"[set, set up, easy, up, to set, easy to, setup...","[This printer is easy to set up., This was eas..."
3,2,101,2_ink_hp_the_you,ink and cartridge,"[ink, hp, the, you, to, cartridges, for, and, ...",[I paid for instant ink for months and everyth...
4,3,96,3_wifi_to_my_the,wifi and connection,"[wifi, to, my, the, it, my wifi, wireless, con...",[But since this one is STILL NOT in wifi conne...
5,4,95,4_tray_paper_paper tray_the paper,paper tray,"[tray, paper, paper tray, the paper, the, slid...",[The only thing I don’t like is the paper tray...
6,5,83,5_service_customer_customer service_he,customer service,"[service, customer, customer service, he, me, ...",[When calling customer support they immediatel...
7,6,78,6_print_slow_to print_printer,print time,"[print, slow, to print, printer, it, the, take...",[memories insufficient on the printer it takes...
8,7,74,7_quality_print quality_quality is_print,quality,"[quality, print quality, quality is, print, pr...",[The print quality is not very good on this on...
9,8,60,8_subscription_ink_the subscription_the,subscription,"[subscription, ink, the subscription, the, for...",[I canceled the subscription and was told I wo...


In [54]:
topic_model.set_topic_labels([
    'outliers',
    'support',
    'set up',
    'ink and cartridge',
    'wifi and connection',
    'paper tray',
    'customer service',
    'print time',
    'quality',
    'subscription',
    'will return',
    'scanner',
    'price',
    'free ink',
    'uninstalling',
    '8015',
    'waste',
    'smart',
    'shipping',
    '*surprise',
    'disappointed ',
    'pages',
    'touch screen',
    'works great',
    'other brands',
    'frustrating ',
    'not recommend',
    'fax',
    'flimsy/sturdy',
    'give stars',
    'jams',
    'at home',
    'lot of issues',
    'print with phone',
    'headaches',
    'worst printer, hate',
    'low cartridge',
    'wanted features',
    'Apple users',
    'printing envelopes',
    'customer service problems',
])

In [58]:
topic_model.visualize_barchart(top_n_topics=4*3, custom_labels=True)

In [59]:
topic_model.save('model_postprocessed.bertopic')

2025-01-18 16:51:28,046 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [50]:
topic_model = BERTopic.load('model.bertopic')

In [62]:
print(corpus[13])
topic_model.visualize_distribution(topic_model.probabilities_[13], min_probability=0.02, custom_labels=True)

I ASSUMMED (my fault) that if i didn't want the instant ink auto delivered that i could just buy ink as needed.


In [63]:
topic_model.visualize_topics()

In [64]:
topic_distr, topic_token_distr = topic_model.approximate_distribution(
    corpus, calculate_tokens=True)
topic_model.visualize_approximate_distribution(corpus[1], topic_token_distr[1])

100%|██████████| 3/3 [00:00<00:00,  7.35it/s]


,It,stopped,working,and,showed,me,message,to,set,up,my,printer
1_set_set up_easy_up,0.000,0.000,0.000,0.000,0.000,0.149,0.450,0.809,1.079,0.931,0.629,0.271
3_wifi_to_my_the,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.122,0.223,0.223,0.223,0.101
9_return_it_return it_wish could,0.114,0.114,0.114,0.114,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [65]:
topic_model.visualize_heatmap( width=850, height=850)

In [66]:
topic_model.visualize_documents(
    corpus,
    embeddings=embeddings,
    hide_annotations=False,
    custom_labels=True,
    width=2000,
    height=1200,
)